# Optimal Portfolios: Sensitivity to Inputs  {#sec-input-sensitivity}

Optimal risky portfolios can be quite sensitive to the inputs, which consist of expected returns, standard deviations, and correlations across assets.  An example is provided in the first section below.  In practice, one must estimate these inputs.  Expected returns are notoriously hard to estimate using historical data, which we discuss in detail in SECTION LINK, so we often resort to models for inputs here or we assume expected returns are equal across assets.  Asset volatilities and correlations can be forecasted more sucessfully.  Forecasting correlations in portfolios with many assets is complicated by the fact that the number of correlations we need to estimate rises quadratically with the number of assets.

## Sensitivity of weights

Let's return to the example of investing in US equity, developed international, and emerging market funds from @sec-optimal-portfolios-theory.  We will fix the input parameters for all but the US equity portfolio and see what happens to the tangency portfolio weights as we vary our assumptions about the US equity portfolio.

### Expected returns {-}

Portfolio optimization is quite sensitive to expected returns.  This is rather unfortunate because financial economists and practitioners are not particularly great at forecasting expected returns.  @fig-sensitivity-expret shows how the tangency portfolio weight changes as a function of different inputs for the US equity expected return.  The tangency portfolio invests nothing in US equity with a US equity expected return input of about 5.5%, but allocates 100% to US equity with an input of about 7.5%.  If the expected return input is 5%, the optimal risky portfolio has short exposure equal to about 50% of total capital.  Thus, a 2.5% difference in expected return input leads to a possible difference in portfolio weight for US equities of 150%!  A 2.5% difference in estimated expected return is not extremely large relative to the precision with which expected returns are estimated.  


In [ ]:
#| label: fig-sensitivity-expret
#| fig-cap: Sensitivity of tangency weights to expected return input
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
import plotly.graph_objects as go
##### Inputs
# Risk-free rate
r = 0.02
# Expected returns
means = np.array([0.06, 0.065, 0.08])
# Standard deviations
sds = np.array([0.15, 0.165, 0.21])
# Correlations
corr12 = 0.75
corr13 = 0.75
corr23 = 0.75
# Covariance matrix
C  = np.identity(3)
C[0, 1] = C[1, 0] = corr12
C[0, 2] = C[2, 0] = corr13
C[1, 2] = C[2, 1] = corr23
cov = np.diag(sds) @ C @ np.diag(sds)

def tangency(means, cov, rf, Shorts):
    n = len(means)
    def f(w):
        mn = w @ means
        sd = np.sqrt(w.T @ cov @ w)
        return -(mn - rf) / sd
    # Initial guess (equal-weighted)
    w0 = (1/n)*np.ones(n)
    # Constraint: fully-invested portfolio
    A = np.ones(n)
    b = 1
    cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
    if Shorts==True:
        # No short-sale constraint
        bnds = [(None, None) for i in range(n)] 
    else:
        # With short-sale constraint
        bnds = [(0, None) for i in range(n)] 
    # Optimization
    wgts_tangency = minimize(f, w0, bounds=bnds, constraints=cons).x
    return wgts_tangency

wgts_true = tangency(means,cov,r,Shorts=True)

# Tangency portfolios for a range of assumed asset 1 expected returns
n = len(means)
num_grid=100
asset1_means = np.linspace(0.04,0.10,num_grid)
wgts = np.zeros((num_grid,n))

for i,m in enumerate(asset1_means):
    wgts[i] = tangency(np.array([m, means[1], means[2]]),cov,r,Shorts=True)
wgt_asset1 = wgts[:,0]
cd = np.empty(shape=(num_grid, n-1,1), dtype=float)
cd[:, 0] = wgts[:,1].reshape(-1, 1)
cd[:, 1] = wgts[:,2].reshape(-1, 1)
string = "Asset 1 Expected Return Input = %{x:0.2%}<br>"
string +="Tangency Portfolio Weights:<br>"
string += "  Asset 1: %{y:0.1%}<br>"
string += "  Asset 2: %{customdata[0]:.1%}<br>"
string += "  Asset 3: %{customdata[1]:.1%}<br>"
string += "<extra></extra>"
trace = go.Scatter(x=asset1_means,y=wgt_asset1,mode='lines', name="Tangency Weight",
    customdata=cd, hovertemplate=string,
)

# Tangency portfolio at assume input
trace_true = go.Scatter(x=[means[0]],y=[wgts_true[0]],mode='markers', name="Tangency Weight at Assumed Input",
    customdata = np.array([[wgts_true[1],wgts_true[2]]]), hovertemplate=string,
)
fig = go.Figure()
fig.add_trace(trace)
# fig.add_trace(trace_true)
fig.layout.xaxis["title"] = "Asset 1 Expected Return Input"
fig.layout.yaxis["title"] = "Asset 1 Tangency Portfolio Weight"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

### Standard Deviations {-}

@fig-sensitivity-sd shows the sensitivity of the US market allocation in the tangency portfolio for different standard deviation inputs.  For values in the region of the historical average volatility of 15%, higher standard deviation inputs result in lower allocations to US equities.


In [ ]:
#| label: fig-sensitivity-sd
#| fig-cap: Sensitivity of tangency weights to standard deviation input
##### Variance input of asset 1:
# Tangency portfolios for a range of assumed asset 1 standard deviations
asset1_sds = np.linspace(0.05,0.25,num_grid)
wgts = np.zeros((num_grid,n))

for i,s in enumerate(asset1_sds):
    sds_new = np.array([s, sds[1], sds[2]])
    wgts[i] = tangency(means,np.diag(sds_new) @ C @ np.diag(sds_new),r,Shorts=True)
wgt_asset1 = wgts[:,0]
cd = np.empty(shape=(num_grid, n-1,1), dtype=float)
cd[:, 0] = wgts[:,1].reshape(-1, 1)
cd[:, 1] = wgts[:,2].reshape(-1, 1)
string = "Asset 1 Standard Deviation Input = %{x}<br>"
string +="Tangency Portfolio Weights:<br>"
string += "  Asset 1: %{y:0.1%}<br>"
string += "  Asset 2: %{customdata[0]:.1%}<br>"
string += "  Asset 3: %{customdata[1]:.1%}<br>"
string += "<extra></extra>"
trace = go.Scatter(x=asset1_sds,y=wgt_asset1,mode='lines', name="Tangency Weight",
    customdata=cd, hovertemplate=string,
)

# Tangency portfolio at assumed input
trace_true = go.Scatter(x=[sds[0]],y=[wgts_true[0]],mode='markers', name="Tangency Weight at Assumed Input",
    customdata = np.array([[wgts_true[1],wgts_true[2]]]), hovertemplate=string,
)
fig = go.Figure()
fig.add_trace(trace)
# fig.add_trace(trace_true)
fig.layout.xaxis["title"] = "Asset 1 Standard Deviation Input"
fig.layout.yaxis["title"] = "Asset 1 Tangency Portfolio Weight"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.55))
fig.show()

### Correlations {-}

@fig-sensitivity-corr shows the sensitivity of the US market's weight in the tangency portfolio for different inputs of the correlation between the US equity fund and the developed international equity fund.  Lower correlations imply greater diversification benefits, so more capital is allocated to US equities for lower correlation levels, all else equal.


In [ ]:
#| label: fig-sensitivity-corr
#| fig-cap: Sensitivity of tangency weights to standard deviation input
##### Correlation of assets 1 and 2:
# Tangency portfolios for a range of assumed asset 1 standard deviations
corr12_grid = np.linspace(0.15,0.95,num_grid)
wgts = np.empty((num_grid,n))

def is_pos_def(x):
    if np.all(np.linalg.eigvals(x) > 0):
        return 'True'
    else:
        return 'False'

for i,c in enumerate(corr12_grid):
    # Covariance matrix
    C  = np.identity(3)
    C[0, 1] = C[1, 0] = c
    C[0, 2] = C[2, 0] = corr13
    C[1, 2] = C[2, 1] = corr23
    # Check feasible correlations
    if is_pos_def(C):
        wgts[i] = tangency(means,np.diag(sds) @ C @ np.diag(sds),r,Shorts=True)
    else:
        print("not positive definite" + str(c*100))
wgt_asset1 = wgts[:,0]
cd = np.empty(shape=(num_grid, n-1,1), dtype=float)
cd[:, 0] = wgts[:,1].reshape(-1, 1)
cd[:, 1] = wgts[:,2].reshape(-1, 1)
string = "Input: Correlation of Assets 1 and 2 = %{x}<br>"
string +="Tangency Portfolio Weights:<br>"
string += "  Asset 1: %{y:0.1%}<br>"
string += "  Asset 2: %{customdata[0]:.1%}<br>"
string += "  Asset 3: %{customdata[1]:.1%}<br>"
string += "<extra></extra>"
trace = go.Scatter(x=corr12_grid,y=wgt_asset1,mode='lines', name="Tangency Weight",
    customdata=cd, hovertemplate=string,
)

# Tangency portfolio at assumed input
trace_true = go.Scatter(x=[corr12],y=[wgts_true[0]],mode='markers', name="Tangency Weight at Assumed Input",
    customdata = np.array([[wgts_true[1],wgts_true[2]]]), hovertemplate=string,
)
fig = go.Figure()
fig.add_trace(trace)
# fig.add_trace(trace_true)
fig.layout.xaxis["title"] = "Correlation of Assets 1 and 2"
fig.layout.yaxis["title"] = "Asset 1 Tangency Portfolio Weight"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.55))
fig.show()

### Error Maximization {-}

The sensitivity of the tangency portfolio to each of these inputs presents a practical problem for investors seeking to implement optimal portfolios in a quantitative sense.  Portfolio optimization is sometimes referred to as error maximization because the tangency portfolio weights may reflect errors in the inputs rather than optimal diversification and risk-reward trade-offs.

When the estimated expected return for an asset is too high relative to its true (but unknown) value, the resulting tangency portfolio will put too much weight on that asset, relative to the unknown *true* tangency portfolio.  Similarly, the tangency portfolio weight will be too low for an asset with an over-estimated variance relative to the tangency portfolio calculated using the true (but unknown) variance.  Finally, optimal portfolios will overallocate capital to assets with underestimated correlations and underallocate capital to assets with overestimated correlations.  The error maximization problem becomes more acute for portfolios with many assets simply because the increase in the number of correlations to be estimated increases the likelihood of substantial errors.

## Estimating the Input List

A simple way to estimate expected return, standard deviations, and correlations is to use historical data.  Specifically, the historical arithmetic average return is an estimate of an asset's expected return.  The historical standard deviation is an estimate of the volatility of the asset's return.  Historical correlations across assets can be estimated using the historical return series of each pair of assets.

To understand the estimation issues that arise in practice, we can simulate the performance of various strategies for using historical data to estimate the input parameters.  We will simulate from an assumed distribution and then use this simulated "historical" data to estimate inputs to our portfolio optimization problem each year.  We assume an investment horizon of 50 years.  We will consider the following strategies for estimating inputs:

* **Est-All**: use historical data to estimate expected returns, standard deviations, and correlations.  The optimal risky portfolio is the tangency portfolio, which is scaled up or down depending on risk aversion.

* **Est-SD-Corr**: use historical data to estimate standard deviations and correlations, and assume expected returns are the same across all assets.  Under this assumption, the optimal risky portfolio is the global minimum variance portfolio.  For the purposes of determining optimal capital allocation based on risk aversion, we use the cross-sectional average of the historical time-series average return.  

* **Est-SD**: use historical data to estimate standard deviations only, and assume correlations across assets are zero and expected returns are the same across all assets and correlations.  This strategy is known as risk parity. As in Est-SD-Corr, we use the cross-sectional average of the historical time-series average return as the expected return input for the purposes of determining optimal capital allocation based on risk aversion. 

* **Est-None**: do not use historical data to estimate expected returns, standard deviations, or correlations. If we assume that all assets have the same expected returns and standard deviations (with zero correlation across assets), the optimal portfolio is an equal-weighted portfolio of the assets. This is also referred to as the $1/N$ portfolio.   For the purposes of determining optimal capital allocation based on risk aversion, we use the cross-sectional averages of the historical time-series return mean and standard deviation as the expected return and standard deviation inputs. 

The four estimation strategies represent differing levels of ambition in terms of using historical data to estimate inputs to the portfolio optimization problem.  Below, we provide some examples to demonstrate potential benefits and costs of attempting to estimate all or part of the input list using historical estimates.


### Length of the Estimation Window {-}

First, let's consider the effect of the length of the estimation window on implementing each of the strategies.  Each year, we'll use the last $T$ years of data to estimate the inputs for a three-asset portfolio, rebalancing our portfolio to the optimal portfolio based on the new input estimates found each year.  We consider estimation window lengths of $T$=10, 20, 30, 40, and 50 years.  We consider two scenarios for the amount of dispersion in true expected returns; the simulations summarized in @fig-input-est-window have more expected return disperion than those described in @fig-input-est-window2.

@fig-input-est-window shows the realized Sharpe ratios of the strategies as a function of the window length.  The displayed values are the averages from 100 simulations.  As a benchmark, we also show the average realized Sharpe ratio of using the true optimal tangency portfolio weights.  Note that these weights assume knowledge of the data-generating process not available to investors in practice.

The figure shows that using a longer time-series to estimate expected returns, standard deviations, and correlations results in better realized performance regardless of the estimation strategy used.^[Recall that Est-None does not use historical estimates in forming portfolio weights within the risky portfolio; it weights risky assets equally. Est-None does rely on cross-sectional averages of historical estimates to allocate capital between the risky portfolio and the risk-free asset.  The improvement with window length for Est-None is due to better capital allocation.]  The improvement is particularly pronounced for Est-All.  This is not surprising as this is the only strategy that uses historical averages as part of the input list.  Sample averages are fairly noisy estimates of population means in small samples, so longer estimation windows are valuable.^[It is worth noting that we are assuming returns are iid (independently and identically distributed) in these simulations.  As such, more data results in better estimates of expected returns (i.e., the population mean).  If expected returns are time-varying, this need not be the case.]


In [ ]:
#| label: fig-input-est-window
#| fig-cap: Simulated performance of various strategies as a function of input estimation window length--more dispersion in expected returns

import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
import plotly.graph_objects as go
from scipy.stats import multivariate_normal as mvn
import pandas as pd

def tangency(means, cov, rf, short_lb):
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''

    n = len(means)
    def f(w):
        mn = w @ means
        sd = np.sqrt(w.T @ cov @ w)
        return -(mn - rf) / sd
    # Initial guess (equal-weighted)
    w0 = (1/n)*np.ones(n)
    # Constraint: fully-invested portfolio
    A = np.ones(n)
    b = 1
    cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
    bnds = [(short_lb, None) for i in range(n)] 
    # Optimization
    wgts_tangency = minimize(f, w0, bounds=bnds, constraints=cons).x
    return wgts_tangency

def gmv(cov, short_lb): 
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''    
    n = len(cov)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if short_lb==None:
        # No position limits
        G = matrix(np.zeros((n,n)), tc="d")
        h = matrix(np.zeros(n), (n, 1), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
        h = matrix(-short_lb * np.ones(n), (n, 1), tc="d")
    # Fully-invested constraint
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b, options={'show_progress': False})
    wgts_gmv = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts_gmv

# Simulation function
def simulation(means, cov, short_lb, seed, window):
	rets = mvn.rvs(means, cov, size=window+T, random_state = seed)
	df = pd.DataFrame(data=rets, columns=['r0','r1','r2'])
	df.columns
	df['mn0']=df['r0'].rolling(window).mean()
	df['mn1']=df['r1'].rolling(window).mean()
	df['mn2']=df['r2'].rolling(window).mean()
	df['sd0']=df['r0'].rolling(window).std()
	df['sd1']=df['r1'].rolling(window).std()
	df['sd2']=df['r2'].rolling(window).std()

	corrs = df[['r0','r1','r2']].rolling(window, min_periods=window).corr()
	df['c01']=corrs.loc[(slice(None),'r0'),'r1'].values
	df['c02']=corrs.loc[(slice(None),'r0'),'r2'].values
	df['c12']=corrs.loc[(slice(None),'r1'),'r2'].values
    
	wgts_true = tangency(means,cov,r,short_lb)
	wgt_cal_true = (wgts_true @ means - r) / (raver * (wgts_true @ cov @ wgts_true))


	model_list = ['true', 'est_none', 'est_all', 'est_sd_corr', 'est_sd']
	for model in model_list:
		df['portret_'+model] = np.nan  # portret is the realized portfolio return of the 100% risky asset portfolio
		if model not in ['true','est_none']:
			df['wgt0_'+model] = np.nan
			df['wgt1_'+model] = np.nan
			df['wgt2_'+model] = np.nan
		df['wgt_cal_'+model] =np.nan
		df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

	for i in np.arange(window,window+T):
		# Full estimation inputs at each point in time
		means = df[['mn0','mn1','mn2']].iloc[i-1].values
		sds   = df[['sd0','sd1','sd2']].iloc[i-1].values
		corr01 = df.loc[i-1,'c01']
		corr02 = df.loc[i-1,'c02']
		corr12 = df.loc[i-1,'c12']
		C  = np.identity(3)
		C[0, 1] = C[1, 0] = corr01
		C[0, 2] = C[2, 0] = corr02
		C[1, 2] = C[2, 1] = corr12
		cov = np.diag(sds) @ C @ np.diag(sds)

		##### Note: all portfolio weights considered to be beginning of period weights
		##### (so multiply by contemporaneous realized returns)
		# Theoretical optimal weights
		model = 'true'
		df.loc[i,'portret_'+model]= df.loc[i,['r0','r1','r2']].values @ wgts_true
		df.loc[i,'raver_portret_'+model] = r + wgt_cal_true * (df.loc[i,'portret_'+model] -r)

		# Full estimation tangency portfolio
		model = 'est_all'
		w0, w1, w2 = tangency(means,cov,r,short_lb)
		df.loc[i,'wgt0_' + model] = w0
		df.loc[i,'wgt1_' + model] = w1
		df.loc[i,'wgt2_' + model] = w2
		# df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
		wgts = np.array([w0, w1, w2])
		df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)

		# Estimate only covariance matrix
		model = 'est_sd_corr'
		w0, w1, w2 = gmv(cov,short_lb)
		df.loc[i,'wgt0_' + model] = w0
		df.loc[i,'wgt1_' + model] = w1
		df.loc[i,'wgt2_' + model] = w2
		# df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
		wgts = np.array([w0, w1, w2])
		df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)


		# Estimate only standard deviations in covariance matrix
		model = 'est_sd'
		cov[0, 1] = cov[1, 0] = 0.0
		cov[0, 2] = cov[2, 0] = 0.0
		cov[1, 2] = cov[2, 1] = 0.0
		w0, w1, w2 = gmv(cov,short_lb)
		df.loc[i,'wgt0_' + model] = w0
		df.loc[i,'wgt1_' + model] = w1
		df.loc[i,'wgt2_' + model] = w2
		# df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
		wgts = np.array([w0, w1, w2])
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))		
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    

		# Equal-weighted portfolio
		model = 'est_none'
		cov[0, 0] = cov[1, 1] = cov[2, 2] = (sds.mean())**2
		wgts = (1/n)*np.ones(n)
		# df.loc[i,'portret_'+model]= df.loc[i,['r0','r1','r2']].values @ wgts_est_none
		df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   



	portret_list = ['raver_portret_' +  model for model in model_list]
	stats = df[portret_list].describe()

	sr_df = pd.DataFrame(dtype=float, columns = ['sr'], index = model_list)
	for model in model_list:
		sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]
		
	return sr_df

## Run for a systematic list of inputs (varying window length)
# Took 1 hour to run 10 parms @ 250 sims each
# Risk aversion
raver = 2
# Risk-free rate
r = 0.02
# Investment period
T = 50

# Number of simulations
num_sims = 100

# Asset Parameters
mns1 = np.array([0.06, 0.10, 0.14])
mns2 = np.array([0.08, 0.10, 0.12])

sds1 = np.array([0.16, 0.20, 0.24])

c1 = 0.75

w1 = 10
w2 = 20
w3 = 30
w4 = 40
w5 = 50

mns_dict = {'mns1':mns1, 'mns2':mns2}
sds_dict = {'sds1':sds1}
corr_dict= {'c1':c1}
window_dict = {'w1': w1, 'w2': w2, 'w3': w3, 'w4': w4, 'w5': w5 }

iterables = [list(mns_dict.keys()),
             list(sds_dict.keys()),
             list(corr_dict.keys()), 
             list(window_dict.keys()),
             np.arange(num_sims)]
idx = pd.MultiIndex.from_product(iterables, names=["means", "sds", "corrs", "window", "sim"])
sim_results = pd.DataFrame(dtype='float', columns=['true', 'est_none', 'est_all', 'est_sd_corr', 'est_sd'], index=idx)

for m in list(mns_dict.keys()):
    means = mns_dict[m]
    n = len(means)
    for s in list(sds_dict.keys()):
        sds = sds_dict[s]
        for c in list(corr_dict.keys()):
            corr12 = corr13 = corr23 = corr_dict[c]
            # Covariance matrix
            C  = np.identity(3)
            C[0, 1] = C[1, 0] = corr12
            C[0, 2] = C[2, 0] = corr13
            C[1, 2] = C[2, 1] = corr23
            cov = np.diag(sds) @ C @ np.diag(sds)

            for w in list(window_dict.keys()):

                # print(m + "\t" + s +  "\t" + c + "\t" + w)

                # Run the simulations
                for sim in range(num_sims):
                    # if np.mod(sim,25)==0:
                        # print('Simulation number: ' + str(sim))
                    sim_results.loc[(m,s,c,w,sim)] = simulation(means, cov, short_lb=None, seed=sim, window=window_dict[w]).T.values

stats = sim_results.groupby(['means', 'sds','corrs','window']).mean()
stats = stats[['true','est_all', 'est_sd_corr', 'est_sd','est_none']]

def compare_plot(mns,sds,corr):
    newdf = stats.loc[(mns,sds,corr,slice(None))].stack().reset_index()
    newdf.columns=['window','strategy','sr']
    label_dict = {'true':'True',
                'est_none': 'Est-None',
                'est_all': 'Est-All',
                'est_sd_corr': 'Est-SD-Corr',
                'est_sd': 'Est-SD'}

    newdf['strategy'] = newdf['strategy'].apply(lambda y: label_dict[y])
    newdf['window'] = newdf['window'].apply(lambda y: window_dict[y])
    # newdf
    import plotly.express as px
    fig = go.Figure()
    fig = px.histogram(newdf, x="strategy", y="sr",
                color='window', barmode='group', histfunc='avg',
                height=400)
    fig.layout.yaxis["title"] = "Sharpe ratio"
    fig.layout.xaxis["title"] = "Strategy"             
    fig.show()

compare_plot('mns1','sds1','c1')

With only 10 years of data, Est-All exhibits an average Sharpe ratio of about the same magnitude as Est-SD and Est-None.  With more data, however, Est-All performs better on average than the other strategies.  This need not always be the case.  For portfolios in which the true dispersion in expected returns is relatively small, as in @fig-input-est-window2, we may need much more data before estimating expected returns using historical means outperforms less ambitious estimation strategies.  For @fig-input-est-window2, even using 50 years of realized returns to estimate expected returns does not result in enough precision for Est-All to outperform Est-SD or Est-None, on average.


In [ ]:
#| label: fig-input-est-window2
#| fig-cap: Simulated performance of various strategies as a function of input estimation window length--less dispersion in expected returns
compare_plot('mns2','sds1','c1')

### Number of Assets {-}

Our ability to accurately estimate the set of inputs to portfolio optimization will also vary with the number of assets under consideration.  In particular, recall that the number of correlations scales quadratically with the number of assets.  We now consider the performance of the four estimation strategies for portfolios of 3, 5, and 10 assets.  We simulate 100 times for each case.  We use an estimation window of 30 years and an investment period of 50 years.   The covariance matrix is scaled such that the theoretical Sharpe ratio is the same for the 3, 5, and 10 asset portfolios.

@fig-input-est-nassets reports the average realized Sharpe ratios across the simulations for each case. The average realized Sharpe ratio of each estimation strategy falls with the number of assets.  The reduction in performance is especially pronounced for Est-SD-Corr, which requires estimation of all pair-wise correlations.  For the 3 asset case, there are 3 correlations to estimate.  This increases to 10 correlations for the 5 asset porfolio and 45 for the 10-asset portfolio.  It is perhaps not surprising that we encounter worse performance when trying to estimate 45 different correlations.  Both Est-All and Est-SD-Corr estimate correlations. For the parameters in this example, estimating expected returns largely in Est-All undoes the poor performance due to estimating correlations found in Est-SD-Corr.


In [ ]:
#| label: fig-input-est-nassets
#| fig-cap: Simulated performance of various strategies as a function of the number of assets
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
import plotly.graph_objects as go
from scipy.stats import multivariate_normal as mvn
import pandas as pd

# Risk aversion
raver = 2

# Risk-free rate
r = 0.02

num_sims = 100
T = 50
win=30


# Simulation function
def simulation(means, cov, short_lb, seed, window):
	rets = mvn.rvs(means, cov, size=window+T, random_state = seed)
	n = len(means)
	return_list = ['r' + str(i) for i in range(n)]
	mean_list  = ['mn' + str(i) for i in range(n)]
	sd_list  = ['sd' + str(i) for i in range(n)]
	corr_list = ['c' + str(i) + str(j) for i in np.arange(n) for j in np.arange(i+1,n)]
	wgt_list = ['wgt' + str(i) for i in range(n)]
	df = pd.DataFrame(data=rets, columns=return_list)

	# Estimate rolling window historical inputs
	for i in np.arange(n):
		df[mean_list[i]] = df[return_list[i]].rolling(window).mean()
	for i in np.arange(n):
		df[sd_list[i]] = df[return_list[i]].rolling(window).std()
	corrs = df[return_list].rolling(window, min_periods=window).corr()
	for i in np.arange(n):
		for j in np.arange(i+1,n):
			df['c'+str(i)+str(j)]=corrs.loc[(slice(None),'r'+str(i)),'r'+str(j)].values
    
	wgts_true = tangency(means,cov,r,short_lb)
	wgt_cal_true = (wgts_true @ means - r) / (raver * (wgts_true @ cov @ wgts_true))


	model_list = ['true', 'est_none', 'est_all', 'est_sd_corr', 'est_sd']
	for model in model_list:
		df['portret_'+model] = np.nan  # portret is the realized portfolio return of the 100% risky asset portfolio
		if model not in ['true','est_none']:
			for wgt in wgt_list:
				df[wgt+'_' + model] = np.nan
		df['wgt_cal_'+model] =np.nan
		df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

	for i in np.arange(window,window+T):
		# Full estimation inputs at each point in time
		means = df[mean_list].iloc[i-1].values
		sds   = df[sd_list].iloc[i-1].values
		C = np.identity(n)
		for i2 in np.arange(n):
			for j in np.arange(i2+1,n):
				C[i2,j] = C[j,i2] = df.loc[i-1,'c'+str(i2)+str(j)]
		cov = np.diag(sds) @ C @ np.diag(sds)

		##### Note: all portfolio weights considered to be beginning of period weights
		##### (so multiply by contemporaneous realized returns)
		# Theoretical optimal weights
		model = 'true'
		df.loc[i,'portret_'+model]= df.loc[i,return_list].values @ wgts_true
		df.loc[i,'raver_portret_'+model] = r + wgt_cal_true * (df.loc[i,'portret_'+model] -r)

		# Full estimation tangency portfolio
		model = 'est_all'
		wgts = tangency(means,cov,r,short_lb)
		for a, wgt in enumerate(wgt_list):
			df.loc[i,wgt+'_'+model] = wgts[a]
		df.loc[i,'portret_'+model] = df.loc[i,return_list].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)

		# Estimate only covariance matrix
		model = 'est_sd_corr'
		wgts = gmv(cov,short_lb)
		for a, wgt in enumerate(wgt_list):
			df.loc[i,wgt+'_'+model] = wgts[a]
		df.loc[i,'portret_'+model] = df.loc[i,return_list].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)


		# Estimate only standard deviations in covariance matrix
		model = 'est_sd'
		for i2 in np.arange(n):
			for j in np.arange(i2+1,n):
				cov[i2,j] = cov[j,i2] =0.0		
		wgts = gmv(cov,short_lb)
		for a, wgt in enumerate(wgt_list):
			df.loc[i,wgt+'_'+model] = wgts[a]
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))		
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'portret_'+model] = df.loc[i,return_list].values @ wgts
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    

		# Equal-weighted portfolio
		model = 'est_none'
		for i2 in np.arange(n):
			cov[i2,i2] = (sds.mean())**2
		wgts = (1/n)*np.ones(n)
		df.loc[i,'portret_'+model] = df.loc[i,return_list].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   



	portret_list = ['raver_portret_' +  model for model in model_list]
	stats = df[portret_list].describe()

	sr_df = pd.DataFrame(dtype=float, columns = ['sr'], index = model_list)
	for model in model_list:
		sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]
		
	return sr_df

# Make theoretical Sharpe ratio constant across cases
mns3 = np.array([0.06, 0.10, 0.14])
mns5 = np.array([0.06, 0.10, 0.14, 0.18, 0.22])
mns10= np.array([0.06, 0.10, 0.14, 0.18, 0.22, 0.06, 0.10, 0.14, 0.18, 0.22])

sds3 = np.array([0.16, 0.20, 0.24])
sds5 = np.array([0.16, 0.20, 0.24, 0.28, 0.32])
sds10= np.array([0.16, 0.20, 0.24, 0.28, 0.32, 0.16, 0.20, 0.24, 0.28, 0.32])

corr = 0.5

mns_dict = {'3':mns3, '5':mns5, '10':mns10}
sds_dict = {'3':sds3, '5':sds5, '10':sds10}

# Adjust covariance matrix so theoretical sharpe ratio is same
sharpes = np.zeros(len(mns_dict.keys()))
for k,key in enumerate(mns_dict.keys()):
    means = mns_dict[key]
    sds   = sds_dict[key]
    n = len(means)
    C = np.identity(n)
    for i in np.arange(0,n):
        for j in np.arange(i+1,n):
            C[i,j] = C[j,i] = corr
    cov = np.diag(sds) @ C @ np.diag(sds)
    wgts_true = tangency(means,cov,r,short_lb=None)
    sr_true = (wgts_true @ means - r) / (np.sqrt(wgts_true @ cov @ wgts_true))
    sharpes[k] = sr_true


iterables = [list(mns_dict.keys()),
             np.arange(num_sims)]
idx = pd.MultiIndex.from_product(iterables, names=["n_assets", "sim"])
sim_results = pd.DataFrame(dtype='float', columns=['true', 'est_none', 'est_all', 'est_sd_corr', 'est_sd'], index=idx)


for k,key in enumerate(mns_dict.keys()):
    # print(key)
    means = mns_dict[key]
    sds   = sds_dict[key]
    n = len(means)
    C = np.identity(n)
    for i in np.arange(0,n):
        for j in np.arange(i+1,n):
            C[i,j] = C[j,i] = corr
    cov = np.diag(sds) @ C @ np.diag(sds)
    cov = cov * (sharpes[k]/sharpes[0])**2

    # Run the simulations
    for sim in range(num_sims):
        # if np.mod(sim,25)==0:
            # print('Simulation number: ' + str(sim))
        sim_results.loc[(key,sim)] = simulation(means, cov, short_lb=None, seed=sim, window=win).T.values

stats = sim_results.groupby(['n_assets']).mean()
stats = stats.reset_index()
stats['num'] = stats['n_assets'].apply(lambda x: int(x))
stats = stats.sort_values('num')
stats = stats[['num','true','est_all', 'est_sd_corr', 'est_sd','est_none']]


# Plot results
import plotly.express as px
newdf = stats.set_index('num').stack().reset_index()
newdf.columns=['# of assets','strategy','sr']
label_dict = {'true':'True',
            'est_none': 'Est-None',
            'est_all': 'Est-All',
            'est_sd_corr': 'Est-SD-Corr',
            'est_sd': 'Est-SD'}

newdf['strategy'] = newdf['strategy'].apply(lambda y: label_dict[y])
fig = go.Figure()
fig = px.histogram(newdf, x="strategy", y="sr",
            color='# of assets', barmode='group', histfunc='avg',
            height=400)
fig.layout.yaxis["title"] = "Sharpe ratio"
fig.layout.xaxis["title"] = "Strategy"             
fig.show()


There are unfortunately not hard-and-fast rules on when to try to estimate expected returns or correlations.  An investor must use their own judgement as to whether the underlying assets should exhibit large enough differences in expected returns to make it beneficial to try to differentiate between them on this dimension.  Similarly, for large portfolios, one must weight the potential diversification benefits of estimating correlations across assets with the potential costs of introducing estimation errors into a high-dimensional covariance matrix.  The benefits are probably more useful for optimization across asset classes than within asset class.  In later chapters, we will describe some methods for addressing both of the estimation issues described above.


## Empirical Performance: Stocks, Bonds, and Gold

We now examine how well we would have done forming optimal portfolios historically with the S&P 500, Treasury bonds, corporate bonds, and gold.  As in the previous section, we consider strategies differing in the extent to which they rely on estimates of expected returns, standard deviations, and correlations.  We use rolling windows of 20 years to estimate each input using historical data.  The data starts in 1968, so our first out-of-sample return is in 1988.  We rebalance the portfolio annually.  Each year, we locate our portfolio along the capital allocation line in oreder to match the expected return of an equal-weighted portfolio (which is the cross-sectional average of the historical average returns).

@fig-stockbondsgold-sharpe shows the realized Sharpe ratio for each strategy.  Est-All underperformed each of the other strategies, including Est-None.  The noisiness in estimating expected returns does not lead to better investment results compared to any of the strategies that assume expected returns are equal across the assets.

Est-SD (risk parity) performs the best in terms of Sharpe ratio in the thirty or so years of implementing each strategy, followed by the Est-SD-Corr and Est-None strategies.  For this set of assets and sample, estimating differences in volatility and correlations across assets was beneficial to performance.  


In [ ]:
#| echo: false
import quandl
quandl.ApiConfig.api_key = "f-5zoU2G4zzHaUtkJ7BY"


def tangency(means, cov, rf, short_lb):
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''

    n = len(means)
    def f(w):
        mn = w @ means
        sd = np.sqrt(w.T @ cov @ w)
        return -(mn - rf) / sd
    # Initial guess (equal-weighted)
    w0 = (1/n)*np.ones(n)
    # Constraint: fully-invested portfolio
    A = np.ones(n)
    b = 1
    cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
    bnds = [(short_lb, None) for i in range(n)] 
    # Optimization
    wgts_tangency = minimize(f, w0, bounds=bnds, constraints=cons).x
    return wgts_tangency
def gmv(cov, short_lb): 
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''    
    n = len(cov)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if short_lb==None:
        # No position limits
        G = matrix(np.zeros((n,n)), tc="d")
        h = matrix(np.zeros(n), (n, 1), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
        h = matrix(-short_lb * np.ones(n), (n, 1), tc="d")
    # Fully-invested constraint
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b, options={'show_progress': False})
    wgts_gmv = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts_gmv    

def optimal(means, cov, raver, rs, rb, short_lb):
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''  
    n = len(cov)
    Q = np.zeros((n + 2, n + 2))
    Q[2:, 2:] = raver * cov
    Q = matrix(Q, tc="d")
    p = np.array([-rs, -rb] + list(-means))
    p = matrix(p, (n + 2, 1), tc="d")
    if short_lb==None:
        # Constraint: short-sales unconstrained, saving weight positive, borrowing weight negative
        G = np.zeros((2, n + 2))
        G[0, 0] = -1
        G[1, 1] = 1
        G = matrix(G, (2, n+2), tc="d")
        h = matrix([0, 0], (2, 1), tc="d")
    else:    
        # Constraint: short-sales constrained, saving weight positive, borrowing weight negative
        G = -np.identity(n + 2)
        G[1, 1] = 1
        G = matrix(G, (n+2, n+2), tc="d")
        h = matrix(np.zeros(n+2), (n+2, 1), tc="d")
    # Constraint: fully-invested portfolio
    A = matrix(np.ones(n+2), (1, n+2), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    # function returns vector of risky asset weights
    return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None    


In [ ]:
#| label: fig-stockbondsgold-sharpe
#| fig-cap: Sharpe ratios of various strategies historically

import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
import plotly.graph_objects as go
from scipy.stats import multivariate_normal as mvn
import pandas as pd

# Pull the data (from sbb.py and gold.py from website codebase)
# Stocks, bonds, bills
nominal = pd.read_csv('https://www.dropbox.com/s/hgwte6swx57jqcv/nominal_sbb.csv?dl=1', index_col=['Year'])

# Gold
d = quandl.get("LBMA/GOLD")['USD (AM)']
gold = d.resample('Y').last().iloc[:-1]
gold.index = [x.year for x in gold.index]
gold.loc[1967] = d.iloc[0]
gold = gold.sort_index().pct_change().dropna()
gold.name = 'Gold'
 

df = pd.concat((nominal, gold), axis=1).dropna()
assets = ['TBills','S&P 500', 'Gold', 'Corporates', 'Treasuries']
df = df[assets]

##### Inputs
# Window length (and initial period)
window = 20
n = len(assets)-1
raver = 5
short_lb = None
T = len(df)-window



# Rolling input estimation
risky = assets[1:]
df.columns = ['rf']+['r'+str(i) for i in range(n)]
asset_list = [str(i) for i in range(n)]
for asset in asset_list:
    df['mn' + asset]=df['r'+asset].rolling(window).mean()
    df['sd' + asset]=df['r'+asset].rolling(window).std()

ret_list = ['r' + asset for asset in asset_list]
corrs = df[ret_list].rolling(window, min_periods=window).corr()

corr_list = []
for j, asset in enumerate(asset_list):
    for k in range(j+1,n):
        df['c'+asset+str(k)]=corrs.loc[(slice(None),'r'+asset),'r'+str(k)].values
df['year'] = df.index
df = df.reset_index()


# Prepare columns for the rolling optimization output
model_list = ['ew', 'est_all', 'est_cov', 'est_sds']
for model in model_list:
    df['portret_'+model] = np.nan      #portret is the realized portfolio return of the 100% risky asset portfolio
    if model not in ['ew']:
        for asset in asset_list:
            df['wgt' + asset + '_' +model] = np.nan
    df['wgt_cal_'+model] =np.nan
    df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

mn_list = ['mn'+asset for asset in asset_list]
sd_list = ['sd'+asset for asset in asset_list] 

# Choose optimal portfolios each time period
for i in np.arange(window,window+T):
    # Full estimation inputs at each point in time
    means = df[mn_list].iloc[i-1].values
    sds   = df[sd_list].iloc[i-1].values
    C  = np.identity(n)
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            C[j, k] = C[k, j] =    df.loc[i-1,'c'+asset+str(k)]  
    cov = np.diag(sds) @ C @ np.diag(sds)

    r = df.loc[i,'rf']
    ##### Note: all portfolio weights considered to be beginning of period weights
    ##### (so multiply by contemporaneous realized returns)
    # Full estimation tangency portfolio
    model = 'est_all'
    wgts = tangency(means,cov,r,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    # df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
    # df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r)/ (wgts @ means - r)
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
    df.loc[i,'expret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (means @ wgts - r)
    df.loc[i,'sd'+model] = df.loc[i,'wgt_cal_'+model] * np.sqrt(wgts @ cov @ wgts)

    # Estimate only covariance matrix
    model = 'est_cov'
    wgts = gmv(cov,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    # df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    # df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = 1.0
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
    df.loc[i,'expret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (means.mean() - r)
    df.loc[i,'sd'+model] = df.loc[i,'wgt_cal_'+model] *np.sqrt(wgts @ cov @ wgts)

    # Estimate only standard deviations in covariance matrix
    model = 'est_sds'
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            cov[j, k] = cov[k, j] = 0.0
    wgts = gmv(cov,short_lb)
    for j, asset in enumerate(asset_list):
        df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    # df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))		
    # df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = 1.0
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    
    df.loc[i,'expret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (means.mean() - r)
    df.loc[i,'sd'+model] = df.loc[i,'wgt_cal_'+model] *np.sqrt(wgts @ cov @ wgts)

    # Equal-weighted portfolio
    model = 'ew'
    for j, asset in enumerate(asset_list):
        cov[j,j] = (sds.mean())**2
    wgts = (1/n)*np.ones(n)
    df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
    # df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    # df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
    df.loc[i,'wgt_cal_'+model] = 1.0
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   
    df.loc[i,'expret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (means.mean() - r)
    df.loc[i,'sd'+model] = df.loc[i,'wgt_cal_'+model] *np.sqrt(wgts @ cov @ wgts)     

# Summarize sharpe ratio, avg ret, sd(ret) for each model
portret_list = ['raver_portret_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
stats = df[portret_list].describe()
sr_df = pd.DataFrame(dtype=float, columns = ['sr','avg_ret','sd_ret'], index = ['est_all', 'est_cov', 'est_sds','ew'])
r = df[np.isnan(df['raver_portret_ew'])==False].rf.mean()
for model in ['est_all', 'est_cov', 'est_sds','ew']:
    sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]
    sr_df.loc[model,'avg_ret'] = stats.loc['mean','raver_portret_' +  model]
    sr_df.loc[model,'sd_ret'] = stats.loc['std','raver_portret_' +  model]

label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'equal weights',
            'est_all': 'estimate all inputs',
            'est_cov': 'estimate covariance matrix only',
            'est_sds': 'estimate standard deviations only'}

xaxis_label_dict = {'true': 'theoretical optimal weights', 
            'ew': 'Est-None',
            'est_all': 'Est-All',
            'est_cov': 'Est-SD-Corr',
            'est_sds': 'Est-SD'}
sr_df = sr_df.reset_index()
sr_df['label'] = sr_df['index'].apply(lambda x: label_dict[x])
sr_df['xaxis_label'] = sr_df['index'].apply(lambda x: xaxis_label_dict[x])


# Plot sharpe ratios
string ="Strategy: %{customdata[0]} <br>"
string += "Sharpe ratio: %{y:0.3f}<br>"
string += "Average return: %{customdata[1]:0.1%}<br>"
string += "SD(return): %{customdata[2]:0.1%}<br>"
string += "<extra></extra>"

fig = go.Figure()
fig.add_trace(go.Bar(x=sr_df['xaxis_label'], y=sr_df['sr'], customdata=sr_df[['label','avg_ret','sd_ret']], hovertemplate=string))
fig.layout.yaxis["title"] = "Sharpe ratio"
fig.layout.xaxis["title"] = "Strategy"
fig.show()

@fig-stockbondsgold-timeseries plots the realized returns for each strategy.  The hoverdata contains the optimal risky portfolio weights as well as the fraction of capital allocated to the risky portfolio.  Since we match the expected return for the Est-None portfolio, only Est-All has time-varying allocation to the optimal all-risky portfolio since it is allows for different expected returns across assets.


In [ ]:
#| label: fig-stockbondsgold-timeseries
#| fig-cap: Returns and weights of various strategies historically
# Plot the time-series of returns and portfolio weights.
for asset in asset_list:
    df['wgt'+asset + '_ew'] = 1/n
       
fig = go.Figure()
for model in ['est_all', 'est_cov', 'est_sds', 'ew']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Year: %{x:4.0f}<br>"
    string += "Return: %{y:0.1%}<br>"
    string += "Weight in Risky Portfolio: %{customdata[0]: 0.1%} <br>"
    string += "Risky Portfolio Weights:<br>"
    string += "  "+ risky[0] +": %{customdata[1]: 0.1%} <br>"
    string += "  "+ risky[1] +": %{customdata[2]: 0.1%} <br>"
    string += "  "+ risky[2] +": %{customdata[3]: 0.1%} <br>"
    string += "  "+ risky[3] +": %{customdata[4]: 0.1%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=df['year'], y=df['raver_portret_'+model], customdata=df[wgt_list], hovertemplate=string, name = xaxis_label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Return"
fig.layout.xaxis["title"] = "Year"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[df['year'].iloc[window], df.year.max()])
fig.show()

## Empirical Performance: Industry Portfolios


In [ ]:
#| echo: false
# Define portfolio class
class portfolio:

    def __init__(self, means, cov, Shorts):
        self.means = np.array(means)
        self.cov = np.array(cov)
        self.Shorts = Shorts
        self.n = len(means)
        if Shorts:
            w = np.linalg.solve(cov, np.ones(self.n))
            self.GMV = w / np.sum(w)
            w = np.linalg.solve(cov, means)
            self.piMu = w / np.sum(w)
        else:
            n = self.n
            Q = matrix(cov, tc="d")
            p = matrix(np.zeros(n), (n, 1), tc="d")
            G = matrix(-np.identity(n), tc="d")
            h = matrix(np.zeros(n), (n, 1), tc="d")
            A = matrix(np.ones(n), (1, n), tc="d")
            b = matrix([1], (1, 1), tc="d")
            sol = Solver(Q, p, G, h, A, b)
            self.GMV = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])

    def frontier(self, m):
        if self.Shorts:
            gmv = self.GMV
            piMu = self.piMu
            m1 = gmv @ self.means
            m2 = piMu @ self.means
            a = (m - m2) / (m1 - m2)
            return a * gmv + (1 - a) * piMu
        else:
            n = self.n
            Q = matrix(self.cov, tc="d")
            p = matrix(np.zeros(n), (n, 1), tc="d")
            G = matrix(-np.identity(n), tc="d")
            h = matrix(np.zeros(n), (n, 1), tc="d")
            A = matrix(np.vstack((np.ones(n), self.means)), (2, n), tc="d")
            b = matrix([1, m], (2, 1), tc="d")
            sol = Solver(Q, p, G, h, A, b)
            return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])

    def tangency(self, r):
        if self.Shorts:
            w = np.linalg.solve(self.cov, self.means - r)
            return w / np.sum(w)
        else:
            def f(m):
                w = self.frontier(m)
                mn = w @ self.means
                sd = np.sqrt(w.T @ self.cov @ w)
                return - (mn - r) / sd
            m = minimize_scalar(f, bounds=[max(r, np.min(self.means)), max(r, np.max(self.means))], method="bounded").x
            return self.frontier(m)

    def optimal(self, raver, rs=None, rb=None):
        n = self.n
        if self.Shorts:
            if (rs or rs==0) and (rb or rb==0):
                Q = np.zeros((n + 2, n + 2))
                Q[2:, 2:] = raver * self.cov
                Q = matrix(Q, tc="d")
                p = np.array([-rs, rb] + list(-self.means))
                p = matrix(p, (n + 2, 1), tc="d")
                G = np.zeros((2, n + 2))
                G[0, 0] = G[1, 1] = -1
                G = matrix(G, (2, n+2), tc="d")
                h = matrix([0, 0], (2, 1), tc="d")
                A = matrix([1, -1] + n*[1], (1, n+2), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None
            else:
                w = np.linalg.solve(self.cov, self.means)
                a = np.sum(w)
                return (a/raver)*self.piMu + (1-a/raver)*self.GMV
        else:
           if (rs or rs==0) and (rb or rb==0):
                Q = np.zeros((n + 2, n + 2))
                Q[2:, 2:] = raver * self.cov
                Q = matrix(Q, tc="d")
                p = np.array([-rs, rb] + list(-self.means))
                p = matrix(p, (n+2, 1), tc="d")
                G = matrix(-np.identity(n + 2), tc="d")
                h = matrix(np.zeros(n+2), (n+2, 1), tc="d")
                A = matrix([1, -1] + n * [1], (1, n+2), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None
           else:
                Q = matrix(raver * self.cov, tc="d")
                p = matrix(-self.means, (n, 1), tc="d")
                G = matrix(-np.identity(n), tc="d")
                h = matrix(np.zeros(n), (n, 1), tc="d")
                A = matrix(np.ones(n), (1, n), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else None

@sec-optimal-portfolios-practice discussed how allowing short sales could result in improvements in mean-variance efficiency.  One example of this was industry portfolios, in which the frontier with short-sales lay entirely to the left of the frontier without shorting in expected return-standard deviation space.  The discussion above, however, suggests that some or all of these efficiency gains may be challenging to obtain out-of-sample given the large number of parameters one needs to estimate for 48 industry portfolios.

To assess this, we run the following exercise.  Using data from 1970 to 2022, we estimate historical average returns, standard deviations, and correlations for 48 industry portfolios each month.  We start in forming portfolios in 1980 in order to have an initial estimation of 120 months of data.  We use expanding windows to estimate inputs.  Based on the estimated inputs each month, we consider the optimal portfolio for a mean-variance investor with risk aversion of 3 that either is or is not short-sale constrained.

@fig-industry-ssc-oos plots the resulting time series of realized returns.  It is clear that when shorting is allowed, the return series is much more volatile.  Less clear from the plot is that the average return is also higher for the portfolio with shorting.  However, the increase in average return is not enough to offset the higher volatility.  The realized Sharpe ratio of the portfolio without short-selling is higher than that of the portfolio with short-selling (0.1573 versus 0.1244, respectively).  Thus, the possible efficiency gains of allowing short-selling would not have materialized for portfolios of industries when historical estimates are used for expected returns, standard deviations, and correlations.


In [ ]:
#| label: fig-industry-ssc-oos
#| fig-cap: Industry Portfolios
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
import plotly.graph_objects as go

SolverOptions["show_progress"] = False

# Read data and clean-up missing data (coded -99.99)
ff48 = pdr("48_Industry_Portfolios", "famafrench", start=1900)[0]

# Clean-up missings
for c in ff48.columns:
    ff48[c] = np.where(ff48[c]==-99.99, np.nan, ff48[c])
ff48 = ff48/100

# Read factor data
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
df = ff48.join(ff3['RF'])
df = df.loc['1970-01':].copy()  # There is missing data prior to 1970

# Estimation window
WINDOW = 120
# Risk aversion 
RAVER = 3


asset_list = ff48.columns
dates = df.index[WINDOW:]
df['retp_ss'] = np.nan
df['retp_noss'] = np.nan
df['retp_ew'] = np.nan
df['sum_risky_ss'] = np.nan
df['sum_risky_noss'] = np.nan
df['sum_risky_ew'] = np.nan

# DataFrame for tracking industry weights
df_wgts = pd.DataFrame(dtype=float, columns = pd.MultiIndex.from_product([['ss','noss','ew'], asset_list]), index=dates)


#The following uses an expanding window to estimate parameters
for d in dates:
    data= df.loc[:d-1,asset_list] 
    mns = data.mean()
    sds = data.std()
    C   = data.corr()
    cov = np.diag(sds)@ C @ np.diag(sds)
    n = len(mns)
    rf = df.loc[d,'RF']
    # print(f'Date: {d}\n')

    # Optimal portfolios w/ shorts
    ss  = portfolio(mns.values,cov.values, Shorts = True)
    wgt_ss = ss.optimal(RAVER, rf, rf)
    if wgt_ss.sum()<0:
        wgt_ss = 0.0*wgt_ss
    df.loc[d,'sum_risky_ss'] = wgt_ss.sum()
    df.loc[d,'retp_ss'] = rf + wgt_ss @ (df.loc[d,asset_list] - rf)
    df_wgts.loc[d,('ss',slice(None))] = wgt_ss
    # print(f'\n\tWith Short-sales:')
    # print(f'\n\tTotal Risky Weight: {wgt_ss.sum(): .2f}\n\tMin Weight: {wgt_ss.min(): .2f}\n\tMax Weight: {wgt_ss.max(): .2f}')

    # Optimal portfolios w/o shorts
    noss  = portfolio(mns.values,cov.values, Shorts = False)
    wgt_noss = noss.optimal(RAVER, rf, rf)
    if wgt_noss.sum()<0:
        wgt_noss = 0.0*wgt_noss    
    df.loc[d,'sum_risky_noss'] = wgt_noss.sum()        
    df.loc[d,'retp_noss'] = rf + wgt_noss @ (df.loc[d,asset_list] - rf)
    df_wgts.loc[d,('noss',slice(None))] = wgt_noss
    # print(f'\n\tWithout Short-sales:')
    # print(f'\n\tTotal Risky Weight: {wgt_noss.sum(): .2f}\n\tMin Weight: {wgt_noss.min(): .2f}\n\tMax Weight: {wgt_noss.max(): .2f}')    

    # Equal-weighted
    mns = mns.mean()*np.ones(n)
    sds = sds.mean()*np.ones(n)
    cov = np.diag(sds) @ np.identity(n) @ np.diag(sds)
    ew = portfolio(mns, cov, Shorts=True)
    wgt_ew = ew.optimal(RAVER,rf,rf)
    if wgt_ew.sum()<0:
        wgt_ew = 0.0*wgt_ew    
    df.loc[d,'retp_ew'] = rf + wgt_ew @ (df.loc[d,asset_list] - rf)
    df.loc[d,'sum_risky_ew'] = wgt_ew.sum() 
    df_wgts.loc[d,('ew',slice(None))] = wgt_ew       
    # print(f'\n\tEqual-weighted:')
    # print(f'\n\tTotal Risky Weight: {wgt_ew.sum(): .2f}\n\tMin Weight: {wgt_ew.min(): .2f}\n\tMax Weight: {wgt_ew.max(): .2f}')    

portrets = df[['retp_ss','retp_noss','retp_ew','RF']+['sum_risky_' + v for v in ['ss','noss','ew']]]
portrets = portrets[portrets.retp_ss.isnull()==False]
portrets['Date'] = portrets.index.to_timestamp('M')

# Plot the time-series of returns (sum of risky portfolio weights as hoverdata)
fig = go.Figure()

label_dict = {'ss': 'With short sales',
            'noss': 'Without short sales'}        

for model in ['ss', 'noss']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Date: %{x}<br>"
    string += "Return: %{y:0.1%}<br>"
    string += "Total Weight in Risky Portfolio: %{customdata: 0.2%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=portrets['Date'], y=portrets['retp_'+model], customdata=portrets['sum_risky_'+model], 
        hovertemplate=string, name = label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Return"
fig.layout.xaxis["title"] = "Date"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()